#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 29)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,WHART,"10,000",11.10,1,P50%,SET,2022-05-30


In [5]:
orders_log = pd.read_csv('../data/orders-log.csv')
orders_log['trans'] = orders_log.apply(lambda row: categorise(row), axis=1)
orders_log['amount'] = orders_log.qty * orders_log.target
orders_log[orders_log.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,AP,-5,P25%,SET100,10000,10.600000,11.100000,0.100000,+0.91%,2,2022-05-10,Buy,106000.000000
16,B,TTB,-6,1L,SET50,100000,1.200000,1.260000,0.020000,+1.61%,2,2022-02-02,Buy,120000.000000
17,S,IP,6,C20.2,mai,7500,20.500000,19.900000,-0.200000,-1.00%,2,2022-02-02,Sell,153750.000000


In [6]:
orders_log.head().sort_values('spd',ascending=False).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,AIT,-3,P25%,SET,"18,000",6.75,6.90,0.15,+2.22%,1,2022-02-02,Buy,"121,500.00"
4,B,BCPG,-3,3L,SET100,"12,000",11.50,11.80,-0.10,-0.84%,1,2022-02-02,Buy,"138,000.00"
1,B,AP,-5,P25%,SET100,"10,000",10.60,11.10,0.10,+0.91%,2,2022-05-10,Buy,"106,000.00"
3,B,BANPU,-6,1L,SET50,"12,000",11.50,12.10,-0.10,-0.82%,1,2022-04-11,Buy,"138,000.00"
2,B,ASK,-15,6L,SET,"3,000",37.75,41.50,0.00,0.00%,0,2022-03-03,Buy,"113,250.00"


### End of Tables in the process

### Print to verify before upload file

In [7]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [8]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [9]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,AP,"10,000",10.60,"106,000.00",P25%,SET100,2,2022-05-10
1,B,TTB,"100,000",1.20,"120,000.00",1L,SET50,2,2022-02-02
2,S,IP,"7,500",20.50,"153,750.00",C20.2,mai,2,2022-02-02


In [10]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    226000.0
S    153750.0
Name: amount, dtype: float64

In [11]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,AP,"10,000",10.60,"106,000.00",P25%,SET100,2,2022-05-10
1,B,TTB,"100,000",1.20,"120,000.00",1L,SET50,2,2022-02-02


In [12]:
cash = 235_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(235000, 226000.0, 9000.0)

In [13]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
2,S,IP,"7,500",20.50,"153,750.00",C20.2,mai,2,2022-02-02


In [14]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(235000, 153750.0, 388750.0)

### After call ord-log (must call everytime that orders change)

In [17]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [18]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [19]:
mb = (df.trans == 'Buy') & (df.spd >= -6)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,AIT,-3,P25%,SET,18000,6.75,6.90,0.15,+2.22%,1,2022-02-02,Buy,121500.0
1,B,AP,-5,P25%,SET100,10000,10.60,11.10,0.10,+0.91%,2,2022-05-10,Buy,106000.0
3,B,BANPU,-6,1L,SET50,12000,11.50,12.10,-0.10,-0.82%,1,2022-04-11,Buy,138000.0
4,B,BCPG,-3,3L,SET100,12000,11.50,11.80,-0.10,-0.84%,1,2022-02-02,Buy,138000.0
6,B,BPP,-3,3L,SET100,9000,15.60,15.90,-0.10,-0.62%,1,2022-02-02,Buy,140400.0
7,B,GLOBAL,-1,1L,SET50,9000,20.60,20.70,0.10,+0.49%,1,2022-02-02,Buy,185400.0
11,B,ORI,-4,RD05pct,SET100,15000,10.20,10.60,0.00,0.00%,1,2022-05-09,Buy,153000.0
15,B,TOP,-4,1L,SET50,3600,56.00,57.00,1.00,+1.79%,1,2022-02-02,Buy,201600.0
16,B,TTB,-6,1L,SET50,150000,1.20,1.26,0.02,+1.61%,2,2022-02-02,Buy,180000.0


In [20]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [21]:
ms = (df.trans == 'Sell') & (df.spd <= 6)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
17,S,IP,6,C20.2,mai,7500,20.5,19.9,-0.20,-1.00%,2,2022-02-02,Sell,153750.0
18,S,NER,4,P50%,SET,9000,7.0,6.8,-0.05,-0.73%,1,2022-04-19,Sell,63000.0
19,S,WHART,6,P50%,SET,10000,11.1,10.5,0.00,0.00%,1,2022-05-30,Sell,111000.0


In [22]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,AIT,-3,P25%,SET,18000,6.75,6.90,0.15,+2.22%,1,2022-02-02,Buy,121500.0
1,B,AP,-5,P25%,SET100,10000,10.60,11.10,0.10,+0.91%,2,2022-05-10,Buy,106000.0
3,B,BANPU,-6,1L,SET50,12000,11.50,12.10,-0.10,-0.82%,1,2022-04-11,Buy,138000.0
4,B,BCPG,-3,3L,SET100,12000,11.50,11.80,-0.10,-0.84%,1,2022-02-02,Buy,138000.0
6,B,BPP,-3,3L,SET100,9000,15.60,15.90,-0.10,-0.62%,1,2022-02-02,Buy,140400.0
7,B,GLOBAL,-1,1L,SET50,9000,20.60,20.70,0.10,+0.49%,1,2022-02-02,Buy,185400.0
11,B,ORI,-4,RD05pct,SET100,15000,10.20,10.60,0.00,0.00%,1,2022-05-09,Buy,153000.0
15,B,TOP,-4,1L,SET50,3600,56.00,57.00,1.00,+1.79%,1,2022-02-02,Buy,201600.0
16,B,TTB,-6,1L,SET50,150000,1.20,1.26,0.02,+1.61%,2,2022-02-02,Buy,180000.0
17,S,IP,6,C20.2,mai,7500,20.50,19.90,-0.20,-1.00%,2,2022-02-02,Sell,153750.0


In [23]:
df[mb | ms].shape[0]

12

In [24]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
8,GULF,Buy,"276,000.00",-7
10,KKP,Buy,"207,000.00",-7
15,TOP,Buy,"201,600.00",-4
7,GLOBAL,Buy,"185,400.00",-1
16,TTB,Buy,"180,000.00",-6


### Select source of orders between these two

In [25]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,AIT,18000,6.75,1,-3,6.90,0.15,+2.22%,P25%,SET,2022-02-02
1,Buy,AP,10000,10.60,2,-5,11.10,0.10,+0.91%,P25%,SET100,2022-05-10
3,Buy,BANPU,12000,11.50,1,-6,12.10,-0.10,-0.82%,1L,SET50,2022-04-11
4,Buy,BCPG,12000,11.50,1,-3,11.80,-0.10,-0.84%,3L,SET100,2022-02-02
6,Buy,BPP,9000,15.60,1,-3,15.90,-0.10,-0.62%,3L,SET100,2022-02-02
7,Buy,GLOBAL,9000,20.60,1,-1,20.70,0.10,+0.49%,1L,SET50,2022-02-02
11,Buy,ORI,15000,10.20,1,-4,10.60,0.00,0.00%,RD05pct,SET100,2022-05-09
15,Buy,TOP,3600,56.00,1,-4,57.00,1.00,+1.79%,1L,SET50,2022-02-02
16,Buy,TTB,150000,1.20,2,-6,1.26,0.02,+1.61%,1L,SET50,2022-02-02
17,Sell,IP,7500,20.50,2,6,19.90,-0.20,-1.00%,C20.2,mai,2022-02-02


In [26]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,AP,10000,10.6,2,-5,11.10,0.10,+0.91%,P25%,SET100,2022-05-10
16,Buy,TTB,150000,1.2,2,-6,1.26,0.02,+1.61%,1L,SET50,2022-02-02
17,Sell,IP,7500,20.5,2,6,19.90,-0.20,-1.00%,C20.2,mai,2022-02-02


In [27]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process